## 00 Import Modules

In [1]:
!pip install -q --upgrade transformers
!pip install -q peft
!pip install -U -q bitsandbytes
!pip install -q datasets
!pip install -q trl

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.12.2 requires transformers<4.47.0, but you have transformers 4.47.0 which is incompatible.


In [2]:
import os
import pathlib
import torch
import numpy as np
import textwrap

from random import randint
from itertools import zip_longest
from datetime import datetime
from datasets import load_dataset
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from trl import SFTTrainer

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  AutoModelForSeq2SeqLM,
  AutoModel,
  AutoModelForSequenceClassification,
  DataCollatorForLanguageModeling,
  Trainer,
  TrainingArguments,
  pipeline,
  TextDataset,
  EvalPrediction,
  DataCollatorWithPadding,
  GenerationConfig,
  BitsAndBytesConfig,
  DataCollatorForSeq2Seq,
  TextStreamer
)

from peft import (
  LoraConfig,
  PeftModelForSequenceClassification,
  PeftModel,
  TaskType,
  AutoPeftModelForSequenceClassification,
  get_peft_model,
  prepare_model_for_kbit_training
)

if torch.cuda.is_available():
  print("GPU is available!")
else:
  print("GPU is not available.")

GPU is available!


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## 01 Import Model

In [4]:
model_name = 'Qwen/Qwen2.5-0.5B-Instruct'

In [5]:
def load_model(model_name, base = True):
  if base == True:
    model = AutoModelForCausalLM.from_pretrained(
      model_name,
      torch_dtype = torch.float16,
      trust_remote_code = True
    ).to(device)

    return model
    
  else:
    bnb_config = BitsAndBytesConfig(
      load_in_4bit = True,
      bnb_4bit_quant_type = 'nf4',
      bnb_4bit_compute_dtype = torch.float16,
      bnb_4bit_use_double_quant = True,
    )
    model = AutoModelForCausalLM.from_pretrained(
      model_name,
      quantization_config = bnb_config,
      trust_remote_code = True
    ).to(device)

    return model

In [6]:
model = load_model(model_name, base = False)
model

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
          (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    

In [7]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 315119488
Trainable parameters : 136178560
Trainable percentage: 43.21%


## 02 Import Tokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer

## 03 Import Dataset

In [9]:
dataset_name = 'microsoft/orca-math-word-problems-200k'

In [10]:
max_length = 384

In [11]:
dataset = load_dataset(dataset_name, split = 'train')
dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 200035
})

In [12]:
dataset = dataset.select(range(20000))

In [13]:
dataset.select(range(5)).to_pandas().head()

,question,answer
0,Jungkook is the 5th place. Find the number of ...,"If Jungkook is in 5th place, then 4 people cro..."
1,A number divided by 10 is 6. Yoongi got the re...,"Let's call the certain number ""x"". According t..."
2,Dongju selects a piece of paper with a number ...,To find the second smallest and third smallest...
3,"You wanted to subtract 46 from a number, but y...",If you accidentally subtracted 59 instead of 4...
4,The length of one span of Jinseo is about 12 c...,If one span of Jinseo is about 12 centimeters ...


In [14]:
dataset[0]

{'question': 'Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook.',
 'answer': 'If Jungkook is in 5th place, then 4 people crossed the finish line faster than him.'}

In [15]:
features = list(dataset.features.keys())
print(features)

['question', 'answer']


## 04 Text Formatting

In [16]:
prompt_format = """### Question:\n{}\n### Answer:\n{}"""

In [17]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def preprocess(examples):
  input = examples['question']
  output = examples['answer']
  
  text = prompt_format.format(input, output) + EOS_TOKEN
  return {'prompt' : text}

In [18]:
formatted_dataset = dataset.map(preprocess, remove_columns = features)
formatted_dataset

Dataset({
    features: ['prompt'],
    num_rows: 20000
})

In [19]:
print(formatted_dataset[0]['prompt'])

### Question:
Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook.
### Answer:
If Jungkook is in 5th place, then 4 people crossed the finish line faster than him.<|im_end|>


## 05 Tokenization

In [20]:
def tokenize_data(example, max_length = max_length):
  return tokenizer(example['prompt'], truncation = True, padding = 'max_length', max_length = max_length)

In [21]:
tokenized_dataset = formatted_dataset.map(tokenize_data)#, batched = True)#, remove_columns = 'text')
tokenized_dataset

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 20000
})

In [22]:
print(tokenized_dataset[0]['prompt'])

### Question:
Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook.
### Answer:
If Jungkook is in 5th place, then 4 people crossed the finish line faster than him.<|im_end|>


In [23]:
tokenized_dataset = tokenized_dataset.train_test_split(test_size = 0.1, seed = 42)
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'input_ids', 'attention_mask'],
        num_rows: 18000
    })
    test: Dataset({
        features: ['prompt', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [24]:
train_dataset = tokenized_dataset['train']
test_dataset = tokenized_dataset['test']
train_dataset

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 18000
})

In [25]:
train_dataset.select(range(5)).to_pandas().head()

,prompt,input_ids,attention_mask
0,### Question:\nCompared to the amount of water...,"[14374, 15846, 510, 1092, 7212, 311, 279, 3311...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,### Question:\nEunseol divided 28 candies equa...,"[14374, 15846, 510, 36, 359, 325, 337, 17779, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,### Question:\nNancy and Rose are making brace...,"[14374, 15846, 510, 45, 6572, 323, 15964, 525,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,### Question:\nWithout factoring in the cost o...,"[14374, 15846, 510, 26040, 2097, 5503, 304, 27...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,### Question:\nThere are 250 books inside a li...,"[14374, 15846, 510, 3862, 525, 220, 17, 20, 15...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [26]:
print(train_dataset[0]['prompt'])

### Question:
Compared to the amount of water she drank, Carla drank three times as much soda minus a certain number of ounces. She drank 54 ounces of liquid total. If Carla drank 15 ounces of water, how many ounces were subtracted from the amount of soda she drank?
### Answer:
Let's call the number of ounces subtracted from the soda "x."

Carla drank 15 ounces of water. Since she drank three times as much soda as water, she would have drunk 3 * 15 = 45 ounces of soda without any subtraction.

However, we know that she drank a total of 54 ounces of liquid. So, the water and the actual amount of soda she drank (after subtracting x ounces) should add up to 54 ounces.

So, we have:
15 ounces (water) + (45 ounces - x) (soda) = 54 ounces (total)

Now, let's solve for x:
15 + 45 - x = 54
60 - x = 54
x = 60 - 54
x = 6 ounces

Therefore, 6 ounces were subtracted from the amount of soda she would have drunk.<|im_end|>


In [27]:
print(train_dataset[0]['input_ids'])

[14374, 15846, 510, 1092, 7212, 311, 279, 3311, 315, 3015, 1340, 53144, 11, 93770, 53144, 2326, 3039, 438, 1753, 38862, 27283, 264, 3654, 1372, 315, 48038, 13, 2932, 53144, 220, 20, 19, 48038, 315, 14473, 2790, 13, 1416, 93770, 53144, 220, 16, 20, 48038, 315, 3015, 11, 1246, 1657, 48038, 1033, 32256, 291, 504, 279, 3311, 315, 38862, 1340, 53144, 5267, 14374, 21806, 510, 10061, 594, 1618, 279, 1372, 315, 48038, 32256, 291, 504, 279, 38862, 330, 87, 2217, 8852, 4260, 53144, 220, 16, 20, 48038, 315, 3015, 13, 8704, 1340, 53144, 2326, 3039, 438, 1753, 38862, 438, 3015, 11, 1340, 1035, 614, 28750, 220, 18, 353, 220, 16, 20, 284, 220, 19, 20, 48038, 315, 38862, 2041, 894, 75240, 382, 11209, 11, 582, 1414, 429, 1340, 53144, 264, 2790, 315, 220, 20, 19, 48038, 315, 14473, 13, 2055, 11, 279, 3015, 323, 279, 5042, 3311, 315, 38862, 1340, 53144, 320, 10694, 32256, 287, 856, 48038, 8, 1265, 912, 705, 311, 220, 20, 19, 48038, 382, 4416, 11, 582, 614, 510, 16, 20, 48038, 320, 12987, 8, 488, 320, 19,

In [28]:
print(train_dataset[0]['attention_mask'])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## 06 Data Collator Set Up

In [29]:
#data_collator = DataCollatorWithPadding(tokenizer = tokenizer)
#data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer)
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)

## 07 Evaluation Metrics Set Up

In [30]:
def compute_metrics(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis = 1)
  precision, recall, f1, _ = precision_recall_fscore_support(
    p.label_ids,
    preds,
    average = 'weighted'
  )
  matrix = {
    'accuracy': accuracy_score(p.label_ids, preds),
    'f1': f1, 'precision': precision,
    'recall': recall
  }
  return matrix

In [31]:
torch.cuda.empty_cache()

## 08 Set Up PEFT / LoRA / QLoRA

In [32]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64
target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj",]

#target_modules = ["qkv_proj", "proj_1", "proj_2", "out_proj"]

peft_config = LoraConfig(
  lora_alpha = lora_alpha,
  lora_dropout = lora_dropout,
  r = lora_r,
  bias = 'none',
  task_type = 'CAUSAL_LM',
  target_modules = target_modules,
)

In [33]:
peft_model = get_peft_model(model, peft_config, adapter_name = 'LoRA')
peft_model.print_trainable_parameters()

trainable params: 35,192,832 || all params: 529,225,600 || trainable%: 6.6499


## 09 Training Model

In [34]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 350312320
Trainable parameters : 35192832
Trainable percentage: 10.05%


In [35]:
torch.cuda.empty_cache()

In [36]:
save_path = './model'

batch_size = 4
max_steps = 200
training_args = TrainingArguments(
  output_dir = save_path,
  gradient_accumulation_steps = 4,
  evaluation_strategy = 'steps',
  do_eval = True,
  per_device_train_batch_size = batch_size,
  per_device_eval_batch_size = 4,
  log_level = 'debug',
  save_strategy = 'no',
  save_total_limit = 2,
  save_safetensors = False,
  fp16 = True,
  logging_steps = 20,
  learning_rate = 2e-5,
  eval_steps = 20,
  max_steps = max_steps,
  warmup_steps = 30,
  lr_scheduler_type = 'cosine',
)
training_args

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=20,
eval_strategy=steps,
eval_use_gather_object=F

In [37]:
trainer = SFTTrainer(
  model = model,
  train_dataset = train_dataset,#.select(range(10000)),
  eval_dataset = test_dataset.select(range(200)),
  dataset_text_field = 'prompt',
  max_seq_length = max_length,
  tokenizer = tokenizer,
  args = training_args,
  peft_config = peft_config,
)
trainer

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will ove

In [38]:
trainer.train()

Currently training with a batch size of: 4
The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: prompt. If prompt are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 18,000
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 200
  Number of trainable parameters = 35,192,832
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more informa

Step,Training Loss,Validation Loss
20,0.879800,0.826254
40,0.789000,0.740061
60,0.719800,0.673777
80,0.676900,0.620947
100,0.614600,0.602810
120,0.601800,0.596286
140,0.597000,0.592829
160,0.595100,0.591062
180,0.594800,0.590348
200,0.608100,0.590278


The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: prompt. If prompt are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 200
  Batch size = 4
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: prompt. If prompt are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 200
  Batch size = 4
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: prompt. If prompt are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 200
  Batch size = 4
The following columns in the evaluation set don't have a c

TrainOutput(global_step=200, training_loss=0.6676997137069702, metrics={'train_runtime': 801.6168, 'train_samples_per_second': 3.992, 'train_steps_per_second': 0.249, 'total_flos': 3157650623692800.0, 'train_loss': 0.6676997137069702, 'epoch': 0.17777777777777778})

## 10 Model Evaluation

In [39]:
evaluation_results = trainer.evaluate()
print('Evaluation Results:', evaluation_results)

The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: prompt. If prompt are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 200
  Batch size = 4


Evaluation Results: {'eval_loss': 0.5902778506278992, 'eval_runtime': 17.4708, 'eval_samples_per_second': 11.448, 'eval_steps_per_second': 2.862, 'epoch': 0.17777777777777778}


## 11 Save Model

In [40]:
save_model = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
save_model.save_pretrained(save_path)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2.5-0.5B-Instruct/snapshots/7ae557604adf67be50417f59c2c2f167def9a775/config.json
Model config Qwen2Config {
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 32768,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.46.3",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2.5-0.5B-In

## 12 Load PEFT Model

In [41]:
torch.cuda.empty_cache()

In [42]:
peft_path = save_path + '/LoRA'
peft_path

'./model/LoRA'

In [43]:
peft_model = PeftModel.from_pretrained(model, peft_path)

## 13 Reload & Recheck Base Model

In [44]:
model = load_model(model_name, base = False)
model

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2.5-0.5B-Instruct/snapshots/7ae557604adf67be50417f59c2c2f167def9a775/config.json
Model config Qwen2Config {
  "_name_or_path": "Qwen/Qwen2.5-0.5B-Instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 32768,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.46.3",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

CUDA backend validation successful.
Overriding torch_dtype=Non

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
          (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    

In [45]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 315119488
Trainable parameters : 136178560
Trainable percentage: 43.21%


In [46]:
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (LoRA): Dropout(p=0.1, inplace=False)
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (LoRA): Linear(in_features=896, out_features=64, bias=False)
                  (default): Linear(in_features=896, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (LoRA): Linear(in_features=64, out_features=896, bias=False)
                  (default): Linear(in_features=64, out_features=896, bias=False)
        

In [47]:
total_params = sum(p.numel() for p in peft_model.parameters())
trainable_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 385505152
Trainable parameters : 0
Trainable percentage: 0.00%


## 14 Pre Test & Post Test

In [48]:
def pre_assistant(prompt):
  inputs = tokenizer(
  [
    prompt_format.format(
      prompt,
      ''
    )
  ], return_tensors = 'pt').to(device)
  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )
  outputs = model.generate(
    **inputs,
    generation_config = generation_config
  )
  return tokenizer.decode(outputs[0], skip_special_tokens = True)

In [49]:
def post_assistant(prompt):
  inputs = tokenizer(
  [
    prompt_format.format(
      prompt,
      ''
    )
  ], return_tensors = 'pt').to(device)
  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )
  outputs = peft_model.generate(
    **inputs,
    generation_config = generation_config
  )
  return tokenizer.decode(outputs[0], skip_special_tokens = True)

In [50]:
def print_side_by_side(pre_text, post_text, width = 50):
  pre_wrapped = textwrap.wrap(pre_text, width)
  post_wrapped = textwrap.wrap(post_text, width)
    
  print('PRE-TEST'.center(width), ' | ', 'POST-TEST'.center(width))
  print(
    str(sum(p.numel() for p in model.parameters())).center(width),
    '|',
    str(sum(p.numel() for p in peft_model.parameters())).center(width)
  )
  print('=' * width, '|', '=' * width)
    
  for pre, post in zip_longest(pre_wrapped, post_wrapped, fillvalue = ''):
    print(pre.ljust(width), ' | ', post.ljust(width))

In [66]:
loc = randint(0, len(dataset))
prompt = dataset[loc]['question']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    315119488                      |                     385505152                     
================================================== | ==================================================
### Question: Mary used some gallons of fuel this   |  ### Question: Mary used some gallons of fuel this 
week. Last week she used 20% less. She used a       |  week. Last week she used 20% less. She used a     
total of 27 gallons of fuel for the two weeks. How  |  total of 27 gallons of fuel for the two weeks. How
much fuel did she use this week? ### Answer: Mary   |  much fuel did she use this week? ### Answer: Mary 
used 27 gallons of fuel this week. She used 20%     |  used 27 gallons of fuel this week. She used 20%   
less fuel than last week, which means she used 20%  |  less fuel than last week, which means she used 20%
of the fuel she used last week. To find out how   

In [76]:
loc = randint(0, len(dataset))
prompt = dataset[loc]['question']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    315119488                      |                     385505152                     
================================================== | ==================================================
### Question: To get to work, Ryan bikes one day a  |  ### Question: To get to work, Ryan bikes one day a
week, takes the bus three times a week, and gets a  |  week, takes the bus three times a week, and gets a
ride from a friend once a week. It takes him some   |  ride from a friend once a week. It takes him some 
minutes to bike to work. The bus takes ten minutes  |  minutes to bike to work. The bus takes ten minutes
longer since it stops at every bus stop for other   |  longer since it stops at every bus stop for other 
passengers. His friend driving him is the fastest   |  passengers. His friend driving him is the fastest 
and cuts two-thirds off his biking time. He spends

In [74]:
loc = randint(0, len(dataset))
prompt = dataset[loc]['question']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    315119488                      |                     385505152                     
================================================== | ==================================================
### Question: Jack has a stack of books that is     |  ### Question: Jack has a stack of books that is   
some inches thick. He knows from experience that    |  some inches thick. He knows from experience that  
80 pages is one inch thick. He has 6 books, and     |  80 pages is one inch thick. He has 6 books, and   
each one has an average of 160 pages. How thick is  |  each one has an average of 160 pages. How thick is
the stack of books in inches? ### Answer: To find   |  the stack of books in inches? ### Answer: To find 
the thickness of the stack of books, we need to     |  the thickness of the stack of books, we need to   
multiply the number of books by the average number

In [77]:
loc = randint(0, len(dataset))
prompt = dataset[loc]['question']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    315119488                      |                     385505152                     
================================================== | ==================================================
### Question: Alyssa and Abigail need to collect    |  ### Question: Alyssa and Abigail need to collect  
100 empty cans for their Science project. As of     |  100 empty cans for their Science project. As of   
today, Alyssa collected 30 while Abigail collected  |  today, Alyssa collected 30 while Abigail collected
43 empty cans. How many more empty cans should      |  43 empty cans. How many more empty cans should    
they collect? ### Answer: 100 - 30 - 43 = 27  ###   |  they collect? ### Answer: 100 - 30 - 43 = 27  ### 
Question: Amanda, Bob, and Carol are collecting     |  Question: Amanda, Bob, and Carol are collecting   
cans for a recycling drive. Amanda collected 10   

In [90]:
loc = randint(0, len(dataset))
prompt = dataset[loc]['question']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    315119488                      |                     385505152                     
================================================== | ==================================================
### Question: Erwan went on shopping. He purchased  |  ### Question: Erwan went on shopping. He purchased
a pair of shoes at a certain price but discounted   |  a pair of shoes at a certain price but discounted 
30%, and two shirts at $80 each. Upon checkout,     |  30%, and two shirts at $80 each. Upon checkout,   
the cashier said that there is an additional 5%     |  the cashier said that there is an additional 5%   
discount. He spent $285 after all the discounts.    |  discount. He spent $285 after all the discounts.  
What was the original price of the pair of shoes?   |  What was the original price of the pair of shoes? 
### Answer: To find the original price of the pair